<a href="https://colab.research.google.com/github/guilherme-argentino/fiap-ia4devs-techchallenge-fase3/blob/main/Fase3_TechChallenge.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Fine-tuning do Modelo BERT com AmazonTitles-1.3MM

Neste notebook, realizaremos o fine-tuning do modelo BERT (`bert-base-uncased`) usando o dataset "The AmazonTitles-1.3MM". O objetivo é treinar o modelo para que ele consiga gerar descrições de produtos com base em seus títulos.

### 1. Instalar dependências


In [1]:
# Instalar as bibliotecas necessárias
!pip install datasets transformers torch pandas

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 471.6/471.6 kB 14.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 8.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 12.3 MB/s eta 0:00:00
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 14.0.2
    Uninstalling pyarrow-14.0.2:
      Successfully uninstalled pyarrow-14.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requires pyarrow<15.0.0a0,>=14.0.1, but you have pyarrow 17.0.0 which is incompatible.


## 2. Importar as Bibliotecas e preparar o Ambiente

In [2]:
import json
import torch
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments
from torch.utils.data import Dataset, DataLoader
import pandas as pd

# Verificar se temos acesso a uma GPU no Colab
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cuda


## 3. Carregar o Tokenizer e o Modelo BERT

In [ ]:
# Carregar o tokenizer BERT
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Carregar o modelo BERT para classificação
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2)
model.to(device)

## 4. Classe Dataset para Gerenciamento de Dados

In [ ]:
class AmazonTitlesDataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)


##5. Leitura em Chunks e Tokenização (para JSONL)
Vamos ajustar a função de leitura para processar arquivos JSONL. Cada linha é um objeto JSON separado, então a função simplesmente lê uma linha por vez e cria blocos (chunks).

In [ ]:
def ler_arquivo_em_blocos_jsonl(caminho_arquivo, tamanho_bloco=10000):
    with open(caminho_arquivo, 'r') as f:
        bloco = []
        for i, linha in enumerate(f):
            bloco.append(json.loads(linha.strip()))  # Lê uma linha como JSON
            if (i + 1) % tamanho_bloco == 0:
                yield bloco
                bloco = []
        if bloco:
            yield bloco

def processar_e_tokenizar_chunk(chunk, max_length=128):
    titles = [item['title'] for item in chunk]
    descriptions = [item['content'] for item in chunk]

    # Concatenar título e descrição
    inputs = [f"{title} [SEP] {description}" for title, description in zip(titles, descriptions)]

    # Tokenização
    encodings = tokenizer(inputs, truncation=True, padding=True, max_length=max_length)

    # Exemplo de rótulos fictícios; substitua conforme necessário
    labels = [1] * len(chunk)

    return encodings, labels


## 6. Configuração do Treinamento


In [ ]:
training_args = TrainingArguments(
    output_dir='./results',          # Diretório de saída para os resultados
    num_train_epochs=3,              # Número de épocas
    per_device_train_batch_size=16,  # Tamanho do batch
    save_steps=1000,                 # Salvar checkpoints a cada 1000 passos
    save_total_limit=2,              # Limite de dois checkpoints salvos
    logging_dir='./logs',            # Diretório de logs
)


## 7. Função de Treinamento por Chunk


In [ ]:
def treinar_com_chunk(encodings, labels):
    dataset = AmazonTitlesDataset(encodings, labels)
    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=dataset,
    )
    # Treina o modelo usando o chunk atual
    trainer.train()


## 8. Baixar dados o Google Drive

Vamos baixar os dados do Google Drive para acessar os arquivos que contêm os dados de treinamento e teste.

In [3]:
!mkdir -p Datasets
!cd Datasets; gdown 12zH4mL2RX8iSvH0VCNnd3QxO4DzuHWnK;
!cd Datasets; unzip LF-Amazon-1.3M.raw.zip; mkdir -p LF-AmazonTitles-1.3M/raw; mv LF-Amazon-1.3M/* LF-AmazonTitles-1.3M

Downloading...
From (original): https://drive.google.com/uc?id=12zH4mL2RX8iSvH0VCNnd3QxO4DzuHWnK
From (redirected): https://drive.google.com/uc?id=12zH4mL2RX8iSvH0VCNnd3QxO4DzuHWnK&confirm=t&uuid=e7e82f98-649f-4167-878e-9adca4b955bd
To: /home/guilherme/Datasets/LF-Amazon-1.3M.raw.zip
100%|████████████████████████████████████████| 890M/890M [01:01<00:00, 14.5MB/s]
Archive:  LF-Amazon-1.3M.raw.zip
   creating: LF-Amazon-1.3M/
  inflating: LF-Amazon-1.3M/lbl.json.gz  
  inflating: LF-Amazon-1.3M/trn.json.gz  
  inflating: LF-Amazon-1.3M/filter_labels_test.txt  
  inflating: LF-Amazon-1.3M/tst.json.gz  
  inflating: LF-Amazon-1.3M/filter_labels_train.txt  


## 9. Processar e Treinar em Chunks
A função principal que faz a leitura do arquivo JSONL em chunks e realiza o fine-tuning do modelo BERT em cada chunk.

In [ ]:
# Processar o arquivo trn.json (JSONL) e realizar o fine-tuning em chunks
for i, chunk in enumerate(ler_arquivo_em_blocos_jsonl('trn.json', tamanho_bloco=10000)):
    print(f"Processando chunk {i+1}")

    encodings, labels = processar_e_tokenizar_chunk(chunk)

    # Executar treinamento no chunk atual
    treinar_com_chunk(encodings, labels)

    print(f"Treinamento com chunk {i+1} completo.")


## 10. Salvar o Modelo Fine-Tuned
Depois de processar todos os chunks e realizar o fine-tuning do modelo, salvamos o modelo treinado.

In [ ]:
# Salvar o modelo fine-tuned e o tokenizer
model.save_pretrained('./1IADT')
tokenizer.save_pretrained('./1IADT')

print("Modelo fine-tuned salvo com sucesso.")
